In [1]:
1e-6/400e-9

2.5

In [3]:
awg = AWG_M8190A_qudi
import hardware.awg.multi_channel_awg_seq as mcas

awg.awg._ch[1].sample_marker_low = 0
awg.awg._ch[1].sample_marker_high = 1.5
AWG_M8190A_qudi.awg._ch[1].output_amplitude = 0.5

In [4]:
awg.awg._ch[1].sync_marker_high=1.5

In [5]:
awg.awg._ch[1].sample_marker_low = 0
awg.awg._ch[1].sample_marker_high = 1.5
AWG_M8190A_qudi.awg._ch[1].output_amplitude = 0.7


seq = mcas.MultiChSeq('Laser', ch_dict = {'awg':[1,2]})
seq.awg =awg
seq.start_new_segment('Starting segment')


seq.asc(length_mus = 10, green=True, orange = True)
# seq.asc(pdawg1=dict(type='sine', frequencies=[1700], amplitudes=[0.99]),
#                 length_mus=10)

seq.sequences['awg'][1].print_info()

0     Laser             10.000000
   0     Starting segment  10.000000
      0                       10.000000wait    1       0       
      1     _missing_smpls_   0.000000wait    0       0       


In [6]:
seq.write_seq()

MCAS 'Laser' has been generated.


In [56]:
seq.run_sequence()

writing sequence Laser on AWG_M8190A_qudi ch 1 took 0.0410 seconds
writing sequence Laser on AWG_M8190A_qudi ch 2 took 0.0270 seconds


In [57]:
seq.stop_awgs()

writing sequence wait on AWG_M8190A_qudi ch 1 took 0.0100 seconds
writing sequence wait on AWG_M8190A_qudi ch 2 took 0.0160 seconds


## Rabi

In [9]:
[1,400e-3]
0.5,200e-3

(0.5, 0.2)

In [10]:
AWG_M8190A_qudi.awg._ch[1].output_amplitude

0.7

In [52]:
awg = AWG_M8190A_qudi
import hardware.awg.multi_channel_awg_seq as mcas

awg.awg._ch[1].sample_marker_low = 0
awg.awg._ch[1].sample_marker_high = 1.5
AWG_M8190A_qudi.awg._ch[1].output_amplitude = 0.7

electron_amp = 0.03 # AU 300mV peak to peak
def awg_electronrabi(taus):
    
    global electron_amp
    
    seq = mcas.MultiChSeq('rabi', ch_dict={'awg': [1, 2]})
#     phases = [0.,180.]
    awg_freq = 1840. # in MHz
#     electron_amp = 0.1 # AU 300mV peak to peak
    oscReadoutPhaseFreq = 0
    nitrogen_transition = 14.0733
    rf_safety = 1
    
    for tau in taus:
        
        seq.start_new_segment('rabi')
        
        seq.asc(pdawg1=dict(type='sine', frequencies=[awg_freq], amplitudes=[electron_amp]),
                length_mus=tau)
        seq = awg_AddWait(seq,5)
        seq = awg_AddReadout(seq)
        seq = awg_AddWait(seq,30) # safety

        
    return seq

def awg_AddReadout(seq):
    length_laserreadout_ns = 3000
    length_lasersafety_ns = 4000
    AOMdelay_ns = 400
    length_laserreadout_smpl = length_laserreadout_ns / 1e3 * 12e3
    AOMdelay_sample = AOMdelay_ns  / 1e3 * 12e3
    
    while length_laserreadout_smpl % 64 != 0:
        length_laserreadout_smpl += 1
        
    seq.asc(length_mus=0.5)  # 2nd step
#     seq.asc(length_mus = 10, green=False, orange = True) #green on, AOM off

    seq.asc(timetagger=False,green=True,orange=False, length_smpl=AOMdelay_sample)
    seq.asc(timetagger=True,green=True,orange=True, length_smpl=length_laserreadout_smpl)    
    seq.asc(timetagger=True,green=False,orange=True, length_smpl=AOMdelay_sample)

    seq.asc(length_mus= length_lasersafety_ns / 1e3)
    return seq

def awg_AddWait(seq, time, orange=False, green=False):
    enable_reuse_segments = False
    if time < 0.:
        for i in range(3): 
            print ('Negative time for sequence in AWG!!!')
    elif time > 10.:
        if int(time / (32 / 12.)) >= 1:
            seq.start_new_segment('wait_loop', reuse_segment=enable_reuse_segments,
                                  loop_count=int(time / (32 / 12.)))
            seq.asc(length_smpl=32000, orange=orange, green=green)
        seq.start_new_segment('wait_additional')
        seq.asc(length_mus=time % (32 / 12.), orange=orange, green=green)
    else:
        seq.asc(length_mus=time, orange=orange, green=green)
    return seq

def AddRfSafety(seq):
    rf_safety = 1
    seq.asc(length_mus=rf_safety)
    return seq

# taus = np.linspace(100,1000,91)
taus = np.arange(0.01,5, 0.02)
# taus = np.arange(.01, 1/electron_amp, 0.015)
# taus = np.linspace(.01, 0.5/electron_amp, 100)
# taus = np.linspace(.01, 1.5, 300)

seqRabi = awg_electronrabi(taus)
seqRabi.awg = awg




# pulsedmasterlogic.set_measurement_settings(invoke_settings=False, 
#                                            controlled_variable= taus,
#                                            number_of_lasers=2*len(taus), 
#                                            laser_ignore_list=[], 
#                                            alternating=True, 
#                                            units=('s', 'arb. u.'))




In [53]:

pulsedmasterlogic.set_measurement_settings(invoke_settings=False, 
                                           controlled_variable= taus*1e-6,
                                           number_of_lasers=len(taus), 
                                           laser_ignore_list=[], 
                                           alternating=False, 
                                           units=('s', 'arb. u.'))

In [54]:
seqRabi.write_seq()

writing sequence wait on AWG_M8190A_qudi ch 1 took 0.0130 seconds
writing sequence wait on AWG_M8190A_qudi ch 2 took 0.0170 seconds
MCAS 'rabi' has been generated.


In [55]:
seqRabi.run_sequence()

writing sequence rabi on AWG_M8190A_qudi ch 1 took 5.4354 seconds
writing sequence rabi on AWG_M8190A_qudi ch 2 took 5.0586 seconds


In [25]:
seqRabi.stop_awgs()

writing sequence wait on AWG_M8190A_qudi ch 1 took 0.0110 seconds
writing sequence wait on AWG_M8190A_qudi ch 2 took 0.0130 seconds


In [709]:
seq.run_sequence()

writing sequence Laser on AWG_M8190A_qudi ch 1 took 0.0140 seconds
writing sequence Laser on AWG_M8190A_qudi ch 2 took 0.0150 seconds


In [218]:
seq.stop_awgs()

writing sequence wait on AWG_M8190A_qudi ch 1 took 0.0120 seconds
writing sequence wait on AWG_M8190A_qudi ch 2 took 0.0120 seconds


In [22]:
awg.awg.write('*IDN?')

Traceback (most recent call last):
  File "C:\Program Files (x86)\Qudi\logic\jupyterkernel\qzmqkernel.py", line 795, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-22-6199190daa35>", line 1, in <module>
    awg.awg.write('*IDN?')
  File "C:\Program Files (x86)\Qudi\hardware\awg\agilent_m8190a.py", line 1096, in write
    raise Exception('Errors occurred while writing to AWG:\n' + errors)
Exception: Errors occurred while writing to AWG:
-410,"Query INTERRUPTED"



In [217]:
12/56

0.21428571428571427

In [56]:
pulsedmasterlogic.fit_container.

In [70]:
awg.awg._ch[1].sample_marker_low = 0
awg.awg._ch[1].sample_marker_high = 1.5
AWG_M8190A_qudi.awg._ch[1].output_amplitude = 0.7


seq = mcas.MultiChSeq('Laser', ch_dict = {'awg':[1,2]})
seq.awg =awg
seq.start_new_segment('Starting segment')


# seq.asc(length_mus = 10, green=True, orange = True)
seq.asc(pdawg1=dict(type='sine', frequencies=[1840], amplitudes=[0.05]),
                length_mus=10)


seq.sequences['awg'][1].print_info()

0     Laser             10.000000
   0     Starting segment  10.000000
      0                       10.000000sine    [1840.] [0.05]  [0.]    0       0       
      1     _missing_smpls_   0.000000wait    0       0       


In [71]:
seq.write_seq()

MCAS 'Laser' has been generated.


In [72]:
seq.run_sequence()

writing sequence Laser on AWG_M8190A_qudi ch 1 took 0.0190 seconds
writing sequence Laser on AWG_M8190A_qudi ch 2 took 0.0190 seconds


In [74]:
seq.stop_awgs()